In [1]:
import os
import scanpy as sc
import pandas as pd
import numpy as np
import json

In [2]:
from spamint import sprout_plus
from spamint import preprocess as pp
import warnings
warnings.filterwarnings("ignore")

/home/wangjingwan/.local/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/wangjingwan/.local/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/wangjingwan/.local/lib/python3.8/site-packages/umap/distances.py:1086: NumbaDeprecatio

# 1. Load files

In [24]:
inputDir = './tutorial/SCC_demo/'
outDir = f'{inputDir}/results/'
sc_exp = pd.read_csv(f'{inputDir}/SC_exp.tsv',sep = '\t',header=0,index_col=0)
sc_meta = pd.read_csv(f'{inputDir}/SC_meta.tsv',sep = '\t',header=0,index_col=0)
st_exp = pd.read_csv(f'{inputDir}/ST_exp.tsv',sep = '\t',header=0,index_col=0)
st_coord = pd.read_csv(f'{inputDir}/ST_coord.csv',sep = ',',header=0,index_col=0)
st_decon = pd.read_csv(f'{inputDir}/ST_decon.tsv',sep = '\t',header=0,index_col=0)
sc_smurf = pd.read_csv(f'{inputDir}/smurf_ref.csv',sep = ',',header=0,index_col=0)
lr_df = pd.read_csv('./LR/human_LR_pairs.txt',sep='\t',header=None)

In [28]:
st_decon.columns = [x.split('sf_')[-1] for x in st_decon.columns]

In [25]:
species = 'Mouse'
st_tp = 'visum'
meta_key = 'level3_celltype'

# Parameters

In [8]:
SUM = 1e4
alpha, beta, gamma, delta, eta = [1, 0.001, 0.001, 0.1, 0.0005]
if st_tp == 'slide-seq':
    num_per_spot = 1
    repeat_penalty = 2
else:
    num_per_spot = 10
    repeat_penalty = int((st_exp.shape[0] * num_per_spot/sc_exp.shape[0]) * 10)

max_rep = 20

In [9]:
from importlib import reload
reload(sprout_plus)

<module 'spamint.sprout_plus' from '/data6/wangjingwan/5.Simpute/SpaMint/spamint/sprout_plus.py'>

# Preprocess

In [26]:
if st_exp.shape[1]<1e4:
    # merfish data, only has 200~500 genes
    sc_adata, st_adata, sc_ref, lr_df = pp.prep_all_adata_merfish(sc_exp = sc_exp, st_exp = st_exp, sc_distribution = sc_smurf, 
                                                        sc_meta = sc_meta, st_coord = st_coord, lr_df = lr_df, SP = species)
else:
    sc_adata, st_adata, sc_ref, lr_df = pp.prep_all_adata(sc_exp = sc_exp, st_exp = st_exp, sc_distribution = sc_smurf, 
                                                            sc_meta = sc_meta, st_coord = st_coord, lr_df = lr_df, SP = species)

Data clean is done! Using 15399 shared genes .


In [30]:
obj_spex = sprout_plus.spaMint(save_path = outDir, st_adata = st_adata, weight = st_decon, 
                 sc_ref = sc_ref, sc_adata = sc_adata, cell_type_key = meta_key, lr_df = lr_df, 
                 st_tp = st_tp)
obj_spex.prep()

sc_ref and sc_adata has different genes, both data are subset to 15399 genes.
Parameters checked!
Getting svg genes
By setting k as 6, each spot has average 5.8408408408408405 neighbors.


# cell selection

In [31]:
sc_agg_meta = obj_spex.select_cells(use_sc_orig = True, p = 0, mean_num_per_spot = num_per_spot, mode = 'strict', max_rep = 1, 
                                    repeat_penalty = repeat_penalty)
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)

	 Estimating the cell number in each spot by the deconvolution result.


KeyboardInterrupt: 

if p != 0, extremely time consuming

In [30]:
# change p to 0.1, use different code to select cells
sc_agg_meta = obj_spex.select_cells(use_sc_orig = True, p = 0.1, mean_num_per_spot = num_per_spot, mode = 'strict', max_rep = 1, 
                                    repeat_penalty = repeat_penalty)
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)

	 Estimating the cell number in each spot by the deconvolution result.
	 SpexMod selects 882 feature genes.
	 Init solution: max - 1.0000,     mean - 0.9614,     min - 0.6215
	 Swap selection start...
	 Swapped solution: max - 1.00,     mean - 0.98,     min - 0.61


In [31]:
alter_sc_exp, sc_agg_meta = obj_spex.gradient_descent(
                alpha, beta, gamma, delta, eta, 
                init_sc_embed = False,
                iteration = max_rep, k = 2, W_HVG = 2,
                left_range = 1, right_range = 2, steps = 1, dim = 2)
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)
with open(f'{outDir}/sc_knn.json', 'w') as fp:
    json.dump(obj_spex.sc_knn, fp)
# utils.save_object(obj_spex, f'{outDir}/obj_spex.pkl')


Running v12 now...
Init sc_coord by affinity embedding...
Avearge shape correlation is: 0.9775563419483628
First-term calculation done!
Second-term calculation done!
Third term calculation done!
Fourth term calculation done!
Hyperparameters adjusted.
-----Start iteration 0 -----
Avearge shape correlation is: 0.9775563419483628
First-term calculation done!
Second-term calculation done!
Third term calculation done!
Fourth term calculation done!
---0 self.loss4 231.45334216469772 self.GAMMA 0.004929331087856589 self.GAMMA*self.loss4 1.1409101549207525
---In iteration 0, the loss is:loss1:1140.91015,loss2:2445.32648,loss3:496.65820,loss4:231.45334,loss5:2943.01440.
---In iteration 0, the loss is:loss1:1140.91015,loss2:1140.91015,loss3:1.14091,loss4:1.14091,loss5:114.09102.
The total loss after iteration 0 is 2398.19315.
-----Start iteration 1 -----
Avearge shape correlation is: 0.978597862997212
First-term calculation done!
Second-term calculation done!
Third term calculation done!
Fourth 